In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
from utils.vector_db import VectorDB, VectorDBConfig
import os
from dotenv import load_dotenv

load_dotenv()
config = VectorDBConfig(api_key=os.getenv('PINECONE_API_KEY'))
vector_db = VectorDB(config)

# Query example
results = vector_db.get_product_recommendations("lipsticks", reformat_results=False, run_reranking=True)
print(results)

2025-02-25 09:28:26,117 - vector_db - INFO - Search success
2025-02-25 09:28:26,495 - vector_db - INFO - Reranking success


RerankResult(
  model='bge-reranker-v2-m3',
  data=[{
    index=0,
    score=0.84733725,
    document={
        brand='L.A. COLORS',
        category_name='Lipstick',
        currency='USD',
        description="Indulge your lips in LA COLORS Moisturizing and Conditioning Cream Push Lipstick. Delivering intense, vibrant color payoff with a single swipe, this ultra-pigmented lipstick ensures your lips stand out with a stunning, eye-catching hue. The creamy &amp; smooth texture glides on smoothly, leaving your lips with a soft, creamy finish that feels comfortable and nourishing. Infused with moisturizing ingredients, this lipstick helps to hydrate and condition your lips, preventing dryness and flakiness for all-day comfort. Enjoy a glossy, high-shine look without any tacky or sticky residue, as the formula delivers a smooth, lightweight feel that won't feel heavy on your lips. These medium-to-full coverage lipsticks are made to be flattering on all skin tones while providing an intense

In [6]:
import random

def buy_product(product_id: int, quantity: int) -> str:
    """
    Process a product purchase and generate a tracking ID for the order.

    Args:
        product_id (int): The unique identifier of the product to purchase
        quantity (int): The number of items to purchase

    Returns:
        str: A confirmation message containing the tracking ID

    Example:
        >>> buy_product(886368088, 2)
        'Order placed successfully with tracking id 8374627463847293847'
    """
    # Generate a long tracking id (Python 3 int is unbounded, emulating a long type)
    tracking_id = random.randint(10**18, 10**19 - 1)
    return f"Order placed successfully with tracking id {tracking_id}"

print(buy_product(886368088, 2))

Order placed successfully with tracking id 3851848649446504755


In [7]:
from pydantic import BaseModel, Field

class Get_Product_Recommendations(BaseModel):
    query_text: str = Field(..., title="query_text", description="The query text for which the product recommendations are requested")

class Buy_Product(BaseModel):
    product_id: int = Field(..., title="product_id", description="The unique Product ID of the product to purchase")
    quantity: int = Field(..., title="quantity", description="The number of items to purchase")

# Get the JSON schema
schema = Get_Product_Recommendations.model_json_schema()
print(schema)


{'properties': {'query_text': {'description': 'The query text for which the product recommendations are requested', 'title': 'query_text', 'type': 'string'}}, 'required': ['query_text'], 'title': 'Get_Product_Recommendations', 'type': 'object'}


In [8]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Any, Dict, Optional, Literal

map_name_to_schema: Dict[str, Any] = {vector_db.get_product_recommendations.__name__: Get_Product_Recommendations}
map_name_to_schema.update({buy_product.__name__: Buy_Product})
map_name_to_func = {func.__name__: func for func in [vector_db.get_product_recommendations, buy_product]}

In [9]:
class Tools:
    def __init__(self):
        self.map_name_to_schema = {}
        self.map_name_to_func = {}
        
    def register(self, func, schema):
        """
        Register a new tool with its schema.
        
        Args:
            func: The function to register
            schema: The pydantic model schema for the function
        """
        self.map_name_to_schema[func.__name__] = schema
        self.map_name_to_func[func.__name__] = func
        
    def get_schema(self, name):
        """Get schema by function name"""
        return self.map_name_to_schema.get(name)
        
    def get_function(self, name):
        """Get function by name"""
        return self.map_name_to_func.get(name)
    
tools = Tools()
tools.register(vector_db.get_product_recommendations, Get_Product_Recommendations)
tools.register(buy_product, Buy_Product)


In [5]:
from openai import pydantic_function_tool
import inspect
import json
from pydantic import BaseModel, Field
from enum import Enum
def generate_openai_schema(func):
  json_schema = pydantic_function_tool(map_name_to_schema[func.__name__])
  # json_schema = map_name_to_schema[func.__name__].model_json_schema()
  json_schema["description"] = inspect.getdoc(func) or "No description available."
  # json_schema["function"]["name"] = func.__name__
  return json_schema

tool_schema = [generate_openai_schema(func) for func in tools.map_name_to_func.values()]
print(json.dumps(tool_schema, indent=2))

NameError: name 'tools' is not defined

In [ ]:
from openai import OpenAI, AzureOpenAI, pydantic_function_tool
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os
load_dotenv()
client = OpenAI(
    api_key = os.getenv('OPENAI_API_KEY')
)
system_prompt = """You are a ReAct(Reasoning and Action Agent) who takes the user query, thinks and reasons about the action required to be taken and then call tools if needed, to answer the user query.

###TASKS:
1. PRODUCT RECOMMENDATION: You have to provide the potential customer with the list of products available based on the customer's query. Ensure that the final output is well-formatted, readable and is easily understandable. Ensure that the final response does not contain any information that is useless to the customer.
2. GENERAL INFORMATION: Taks that don't require product information. Provide answers politely in such cases.
###OUTPUT FORMAT: Always make sure that your final output is readable and is easily understandable."""
messages = [
      {"role": "system", "content": system_prompt},
     {"role": "user", "content": "Give me the lipsticks with the highest discounts"}]

def call_llm(messages: Dict[str, Any]):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages,
    tools=tools,
    tool_choice="auto",
    temperature=0,
    parallel_tool_calls=True
  )
  return response

response_message = call_llm(messages).choices[0].message
content = response_message.content
tool_calls = response_message.tool_calls

In [62]:
tool_calls

[ChatCompletionMessageToolCall(id='call_N0oZTSxbUiB8XqyJ3OY9j1ou', function=Function(arguments='{"query_text":"lipsticks with the highest discounts"}', name='Get_Product_Recommendations'), type='function')]

In [56]:
def call_function(name, args):
   return map_name_to_func[name](**args)

In [57]:
import json

if content:
    print("### Model Response:\n")
    print(f"```\n{content}\n```")
elif tool_calls:
    for tool_call in tool_calls:
        name = tool_call.function.name.lower()
        args = json.loads(tool_call.function.arguments)
        messages.append({"role": "assistant", "tool_calls": [tool_call.model_dump()]})
        result = call_function(name, args)

        print("### Tool Result:\n")
        print(f"```\n{result}\n```")

        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(result)
        })

    response_message = call_llm(messages).choices[0].message
    content = response_message.content

    print("### After Tool Response:\n")
    print(f"```\n{content}\n```")


### Model Response:

```
Hello! How can I assist you today?
```


In [1]:
from agent import Agent
agent = Agent()

In [7]:
# user_query = "Give me the price difference between Floor mats and cooling sheets!"
# user_query = "I want 1 of each of the floor mat with the highest cost and the cooling sheet with the lowest cost!"
# user_query = "Give me the list of lipsticks with the highest discounts!"
# user_query = "Buy 1 qty of the lipstick with the highest discount"
user_query = "List out lipsticks"
user_query = "Buy 1 qty only of the lipstick with the highest discount"
session_id = " e168ce7f-873a-4854-a514-9378cf0ee5a3" #"abc"
agent_response, session_id = agent.run(session_id=session_id, user_query=user_query)

2025-02-25 14:33:09,571 - agent - DEBUG - Loaded existing session:  e168ce7f-873a-4854-a514-9378cf0ee5a3
2025-02-25 14:33:09,574 - agent - DEBUG - Processing user query: Buy 1 qty only of the lipstick with the highest discount
2025-02-25 14:33:10,814 - agent - DEBUG - Tool called: buy_product with arguments: {'product_id': 33685485, 'quantity': 1}
2025-02-25 14:33:10,815 - agent - DEBUG - Response from tool buy_product: Order with product_id: 33685485 of quantity: 1 has been added to the cart ✅
2025-02-25 14:33:12,071 - agent - DEBUG - Response from LLM: Your order for 1 quantity of the **Maybelline Color Sensational Elixir Lip Lacquer, Caramel Infused** has been successfully added to the cart. If you need any further assistance, feel free to ask!
2025-02-25 14:33:12,073 - agent - DEBUG - Generated response: Your order for 1 quantity of the **Maybelline Color Sensational Elixir Lip Lacquer, Caramel Infused** has been successfully added to the cart. If you need any further assistance, f

[{'role': 'system', 'content': "You are a ReAct(Reasoning and Action Agent) who accepts the user query, thinks and reasons about the action required to be taken and then call tools if needed while adhering to the strict instructions provided, to answer the user query.\nIf required you may also reason and decompose the main task into subtasks and then call the tools to perform the subtasks to provide the best possible answer.\n\n###TASKS:\n1. PRODUCT RECOMMENDATION: You have to provide the potential customer with the list of products available based on the customer's query. Do not provide useless information like Product ID to the customer in the final output.\n2. BUY PRODUCT: Given the product_id and the quantity of the product_id required, you can place an order for the product.\n2. GENERAL INFORMATION: Taks that don't require product information. Provide answers politely in such cases.\n\n### INSTRUCTIONS:\n1. Always ask the user for the quantity required, if you do not have adequate

In [ ]:
# I want two quantities of the best rated product that is good for my skin!

In [9]:
session_id

'7f6d5d56-7db4-44ad-a504-117b7bfb9a65'

In [8]:
from IPython.display import display, Markdown, Latex
display(Markdown(agent_response))

Your order for 1 quantity of the **Maybelline Color Sensational Elixir Lip Lacquer, Caramel Infused** has been successfully added to the cart. If you need any further assistance, feel free to ask!

In [ ]:
system_prompt = """
You are an intelligent query re-writer and a query descomposer that rewrites the user query based on the previous conversation to make the query more clear and concise. You can also decompose the query into smaller parts to understand the user's intent better.

###TASKS:
1. Query-Rewriting: Rewrite the user_query based on the previous conversation to make it more clear and precise.
2. Query-Decomposition: Decompose the user_query into smaller parts to understand the user's intent better keeping in mind that they are useful when the queries are being used by the tools.

### INSTRUCTIONS:
- Note that you have two tools with you:
a. Get_Product_Recommendations: This is a tool that provides product recommendations based on the user query.
b. Buy_Product: This is a tool that processes a product purchase, given the product_id and quantity.

### OUTPUT FORMAT:
In each of the cases, the output should be a properly formatted **python list**.


### EXAMPLES:
1. Query-Decomposition:
    - Input: "I want two items of the best rated product for household cleaning!"
    -Reasoning: The user wants to buy two quantities of the best rated product for household cleaning, that consists of two smaller queries-"G" So I can decompose the Input as follows:
    - Output: ["best rated product", 2]"""


In [2]:
import requests
import json

# Lambda Function URL
url = "https://bbjrxvwdo6odnztdxezroknyqa0kakcz.lambda-url.ap-south-1.on.aws/"

# Request payload
payload = {
    "user_query": "give me a list of lipsticks"
    # "session_id": "abc123"  # Optional: Can be None if you want a new session
}

# Sending a POST request
response = requests.post(url, json=payload)

# Print response
print("Status Code:", response.status_code)
# print("Response:", response.json())
agent_response = response.json()['response']
from IPython.display import display, Markdown, Latex
display(Markdown(agent_response))


Status Code: 502


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
response.json()

{'user_query': 'give me a list of lipsticks',
 'response': "Here are some lipsticks you might be interested in:\n\n1. **L.A. Colors Long-lasting Moisture Cream Velvet Plush Lipstick, Exquisite**\n   - **Price:** $5.94\n   - **Rating:** 4.3\n   - **Description:** This ultra-pigmented lipstick delivers intense color payoff with a creamy texture that glides on smoothly. Infused with moisturizing ingredients, it hydrates and conditions your lips for all-day comfort.\n\n2. **Revlon ColorStay Overtime Longwearing Gloss Lipstick with Vitamin E, 270 Relentless Raisin**\n   - **Price:** $9.47\n   - **Rating:** 4.3\n   - **Description:** Lasting up to 16 hours, this colorful lipstick stays vibrant without smudging or feathering. It features a shiny top coat for added moisture and shine, with 99% moisturizing ingredients.\n\n3. **Maybelline Color Sensational Elixir Lip Lacquer, Caramel Infused**\n   - **Price:** $11.19\n   - **Rating:** 4.0\n   - **Description:** This lip lacquer offers saturated

In [3]:
!uv pip install polyline


Using Python 3.12.8 environment at: E:\Programs\Pythonenv\pytorchenv
Resolved 1 package in 427ms
Prepared 1 package in 26ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 19ms
 + polyline==2.0.2


In [6]:
from typing import List, Optional
from pydantic import BaseModel, Field

def checkout(product_id: int, quantity: int, price: float, discount: Optional[float] = 0.0) -> float:
    """Calculates the price of a single item after applying discount."""
    return quantity * (price - discount)

class CartItem(BaseModel):
    product_id: int = Field(..., description="Unique identifier for the product")
    quantity: int = Field(..., gt=0, description="Number of items purchased")
    product_price: float = Field(..., gt=0, description="Price per unit of the product")
    discount: Optional[float] = Field(0.0, ge=0, description="Discount on the product")

class CartData(BaseModel):
    cart_items: List[CartItem] = Field(..., description="List of items in the cart")
    shipment_distance: float = Field(..., ge=0, description="Distance for shipment")

def calculate_total_price(data: CartData) -> float:
    """Calculates the total price of the items in the cart including shipment cost."""
    total_price = 0.0
    
    for item in data.cart_items:
        total_price += checkout(item.product_id, item.quantity, item.product_price, item.discount)
    
    total_price += data.shipment_distance * 0.1
    return total_price

# Example Usage:
cart_data = CartData(
    cart_items=[
        CartItem(product_id=1, quantity=2, product_price=50.0, discount=5.0),
        CartItem(product_id=2, quantity=1, product_price=100.0)
    ],
    shipment_distance=20.0
)

print(calculate_total_price(cart_data))
pydantic_function_tool(CartData)

192.0


{'type': 'function',
 'function': {'name': 'CartData',
  'strict': True,
  'parameters': {'$defs': {'CartItem': {'properties': {'product_id': {'description': 'Unique identifier for the product',
       'title': 'Product Id',
       'type': 'integer'},
      'quantity': {'description': 'Number of items purchased',
       'exclusiveMinimum': 0,
       'title': 'Quantity',
       'type': 'integer'},
      'product_price': {'description': 'Price per unit of the product',
       'exclusiveMinimum': 0.0,
       'title': 'Product Price',
       'type': 'number'},
      'discount': {'anyOf': [{'minimum': 0.0, 'type': 'number'},
        {'type': 'null'}],
       'default': 0.0,
       'description': 'Discount on the product',
       'title': 'Discount'}},
     'required': ['product_id', 'quantity', 'product_price', 'discount'],
     'title': 'CartItem',
     'type': 'object',
     'additionalProperties': False}},
   'properties': {'cart_items': {'description': 'List of items in the cart',
     